# Item-based Models

In this file we are going to proceed with the building in the models. Machine learning models to build recommender systems. The models that are going to be built are Collaborative filtering using Item-based rating prediction (ItemKNN) and Item-based classification (ItemKNN).

Collaborative filtering is a technique used in recommendation systems to predict or classify items based on the preferences or behavior of similar users or items. Item-based Collaborative Filtering (CF) focuses on the similarity between items rather than users. There are two main approaches within item-based CF: rating prediction and classification.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

In [2]:
# We import the necessary libraries
import pandas as pd
import numpy as np
import os

In [3]:
# We print the directory where the file is located
print(os.getcwd())

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


In [4]:
# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'cleaned'))

['final_sample30_parquet',
 'final_sample50_parquet',
 'final_sample5_parquet',
 'movielens_parquet',
 'netflix_parquet',
 'sample_tenth_netflix',
 'unpacked_reviews_df_100k.parquet']

In [5]:
# We read the final_sample file and store it in a dataframe
df = pd.read_parquet('cleaned/sample_tenth_netflix')

In [6]:
# We print shape of the dataframe
df.shape

(143, 4)

In [7]:
# We print the first 5 rows of the dataframe
df.head()

,movieId,year,title,review_data
123,124,2000,Cold Blooded,"[{'date': 2002-04-01, 'rating': 3.0, 'userId':..."
1192,13434,1998,Beast Cops,"[{'date': 2003-02-20, 'rating': 2.0, 'userId':..."
462,4601,1994,The Scout,"[{'date': 2003-12-27, 'rating': 3.0, 'userId':..."
351,352,1997,Pick a Card,"[{'date': 2003-10-13, 'rating': 4.0, 'userId':..."
1057,9560,2000,New World Disorder,"[{'date': 2003-07-06, 'rating': 4.0, 'userId':..."


In [8]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'year', 'title', 'review_data'], dtype='object')

In [9]:
# We check if there are movies with less than 10 reviews, if there are we take them out to handle the cold start problem
df = df[df['review_data'].apply(lambda x: len(x) if x is not None else 0) > 10]

In [10]:
# Step 1: Explode the review_data column
exploded_df = df.explode('review_data').reset_index(drop=True)


In [11]:
exploded_df.head()

exploded_df.shape

# We get the first row of exploded_df
exploded_df.iloc[0]

,movieId,year,title,review_data
0,124,2000,Cold Blooded,"{'date': 2002-04-01, 'rating': 3.0, 'userId': ..."
1,124,2000,Cold Blooded,"{'date': 2002-05-14, 'rating': 3.0, 'userId': ..."
2,124,2000,Cold Blooded,"{'date': 2001-06-06, 'rating': 3.0, 'userId': ..."
3,124,2000,Cold Blooded,"{'date': 2001-07-28, 'rating': 2.0, 'userId': ..."
4,124,2000,Cold Blooded,"{'date': 2002-02-04, 'rating': 1.0, 'userId': ..."


(799645, 4)

movieId                                                      124
year                                                        2000
title                                               Cold Blooded
review_data    {'date': 2002-04-01, 'rating': 3.0, 'userId': ...
Name: 0, dtype: object

In [12]:
# Convert the exploded dictionary column into separate columns
expanded_review_data = pd.json_normalize(exploded_df['review_data'])


In [13]:
expanded_review_data.head()

,date,rating,userId
0,2002-04-01,3.0,68845
1,2002-05-14,3.0,349975
2,2001-06-06,3.0,1619594
3,2001-07-28,2.0,2377588
4,2002-02-04,1.0,1361044


In [14]:
# Concatenate the expanded data with the original DataFrame
df = pd.concat([exploded_df.drop('review_data', axis=1), expanded_review_data], axis=1)

# Verify the shape of the final DataFrame
print("DataFrame shape:", df.shape)


DataFrame shape: (799645, 6)


In [15]:
df.head()

,movieId,year,title,date,rating,userId
0,124,2000,Cold Blooded,2002-04-01,3.0,68845
1,124,2000,Cold Blooded,2002-05-14,3.0,349975
2,124,2000,Cold Blooded,2001-06-06,3.0,1619594
3,124,2000,Cold Blooded,2001-07-28,2.0,2377588
4,124,2000,Cold Blooded,2002-02-04,1.0,1361044


In [16]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())


254342
143


In [17]:
# Check if there are userIDs that haven't rated movies at all
print(df['userId'].value_counts().sort_values(ascending=False))

# Check if there are movieIDs that haven't been rated at all
print(df['movieId'].value_counts().sort_values(ascending=False))

userId
305344     142
387418     141
2439493    133
1664010    130
2118461    125
          ... 
1093522      1
401886       1
1753823      1
1707129      1
773753       1
Name: count, Length: 254342, dtype: int64
movieId
30       118413
4705      58536
290       47651
4727      41128
199       35509
          ...  
9249         83
9603         83
9372         71
13478        46
13405        29
Name: count, Length: 143, dtype: int64


In [18]:
# Check if there are NaN values in the rating column
df['rating'].isna().sum()

0

In [19]:
# Group the DataFrame by 'userId' and count the number of ratings per user
user_rating_counts = df.groupby('userId').size()

# Filter out users who have rated less than 5 movies
active_users = user_rating_counts[user_rating_counts >= 5].index.tolist()

# Select rows from the original DataFrame for active users
df = df[df['userId'].isin(active_users)]

# Display the filtered DataFrame
df.head()


,movieId,year,title,date,rating,userId
0,124,2000,Cold Blooded,2002-04-01,3.0,68845
4,124,2000,Cold Blooded,2002-02-04,1.0,1361044
6,124,2000,Cold Blooded,2005-08-06,3.0,1716677
10,124,2000,Cold Blooded,2001-12-20,2.0,1521983
12,124,2000,Cold Blooded,2001-12-07,2.0,650960


In [20]:
df.shape

(424670, 6)

In [21]:
# We take a sample of 10% of the data
df = df.sample(frac=0.05, random_state=42)

In [22]:
# Verify the shape of the final DataFrame
print("DataFrame shape:", df.shape)

DataFrame shape: (21234, 6)


In [23]:
# We move to the parquets folder to read the file with the Netflix genres

# We print the directory where the file is located
print(os.getcwd())

# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'parquets'))

# We read the netflix_genres file and store it in a dataframe
df_genres = pd.read_csv('parquets/netflix_genres.csv')

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


['movies_df', 'movie_title_df', 'netflix_df', 'netflix_genres.csv', 'users_df']

In [24]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())

17037
139


In [25]:
# Merge the two DataFrames on the 'movieId' column, keeping all rows from the left DataFrame
df = pd.merge(df, df_genres, on='movieId', how='left')

print(df)


       movieId  year                                title        date  rating  \
0          290  2004  Harold and Kumar Go to White Castle  2005-05-22     1.0   
1         4506  1961               Breakfast at Tiffany's  2004-04-13     4.0   
2         4727  1979                 Apocalypse Now Redux  2004-05-25     5.0   
3        13629  1951                  Alice in Wonderland  2004-08-13     3.0   
4         4727  1979                 Apocalypse Now Redux  2003-11-28     5.0   
...        ...   ...                                  ...         ...     ...   
21229     4506  1961               Breakfast at Tiffany's  2005-11-08     3.0   
21230      199  1978                      The Deer Hunter  2004-07-05     3.0   
21231     4805  1993                         True Romance  2004-01-04     3.0   
21232     4761  1981                   The Cannonball Run  2005-05-06     3.0   
21233      311  1994                              Ed Wood  2005-09-12     4.0   

        userId             

In [26]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())

17037
139


In [27]:
# We split the content of the genres by the '|'
df['genres'] = df['genres'].str.split('|')

print(df['genres'])

0                                     [Adventure, Comedy]
1                                [Comedy, Drama, Romance]
2                                   [Drama, Mystery, War]
3        [Animation, Adventure, Family, Fantasy, Musical]
4                                   [Drama, Mystery, War]
                               ...                       
21229                            [Comedy, Drama, Romance]
21230                                        [Drama, War]
21231                   [Crime, Drama, Romance, Thriller]
21232                             [Action, Comedy, Sport]
21233                          [Biography, Comedy, Drama]
Name: genres, Length: 21234, dtype: object


In [28]:
# One-hot encode the 'genres' column
df = df.join(df['genres'].str.join('|').str.get_dummies())

# Drop the original 'genres' column
df.drop('genres', axis=1, inplace=True)

print(df)


       movieId  year                                title        date  rating  \
0          290  2004  Harold and Kumar Go to White Castle  2005-05-22     1.0   
1         4506  1961               Breakfast at Tiffany's  2004-04-13     4.0   
2         4727  1979                 Apocalypse Now Redux  2004-05-25     5.0   
3        13629  1951                  Alice in Wonderland  2004-08-13     3.0   
4         4727  1979                 Apocalypse Now Redux  2003-11-28     5.0   
...        ...   ...                                  ...         ...     ...   
21229     4506  1961               Breakfast at Tiffany's  2005-11-08     3.0   
21230      199  1978                      The Deer Hunter  2004-07-05     3.0   
21231     4805  1993                         True Romance  2004-01-04     3.0   
21232     4761  1981                   The Cannonball Run  2005-05-06     3.0   
21233      311  1994                              Ed Wood  2005-09-12     4.0   

        userId  Action  Adv

In [29]:
df.head

<bound method NDFrame.head of        movieId  year                                title        date  rating  \
0          290  2004  Harold and Kumar Go to White Castle  2005-05-22     1.0   
1         4506  1961               Breakfast at Tiffany's  2004-04-13     4.0   
2         4727  1979                 Apocalypse Now Redux  2004-05-25     5.0   
3        13629  1951                  Alice in Wonderland  2004-08-13     3.0   
4         4727  1979                 Apocalypse Now Redux  2003-11-28     5.0   
...        ...   ...                                  ...         ...     ...   
21229     4506  1961               Breakfast at Tiffany's  2005-11-08     3.0   
21230      199  1978                      The Deer Hunter  2004-07-05     3.0   
21231     4805  1993                         True Romance  2004-01-04     3.0   
21232     4761  1981                   The Cannonball Run  2005-05-06     3.0   
21233      311  1994                              Ed Wood  2005-09-12     4.0  

In [30]:
# We have to rename all the genre columns and make them start with 'genre_'
df.columns

genre_columns = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 
        'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller',
       'War']

# Define the new column names with 'genre_' prefix
new_columns = {'{}'.format(col): 'genre_{}'.format(col) for col in genre_columns}

# Rename the columns in the DataFrame
df.rename(columns=new_columns, inplace=True)

# Now, df will have the genre columns starting with 'genre_'
print(df.columns)


Index(['movieId', 'year', 'title', 'date', 'rating', 'userId', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller',
       'War'],
      dtype='object')

Index(['movieId', 'year', 'title', 'date', 'rating', 'userId', 'genre_Action',
       'genre_Adventure', 'genre_Animation', 'genre_Biography', 'genre_Comedy',
       'genre_Crime', 'genre_Documentary', 'genre_Drama', 'genre_Family',
       'genre_Fantasy', 'genre_Film-Noir', 'genre_History', 'genre_Horror',
       'genre_Music', 'genre_Musical', 'genre_Mystery', 'genre_Romance',
       'genre_Sci-Fi', 'genre_Short', 'genre_Sport', 'genre_Thriller',
       'genre_War'],
      dtype='object')


In [31]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())

17037
139


# Item-based Rating Prediction (ItemKNN)

### Step 1: User-Item matrix construction
The first thing we have to do is build the user-item matrix:

- Choose a similarity metric to calculate the similarity between items. Common metrics include cosine similarity, Pearson correlation coefficient, and Jaccard similarity.
- Calculate the similarity between each pair of items based on the ratings provided by users. This will result in an item-item similarity matrix.

#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.

##### Libraries Used
- **pandas**: A powerful data manipulation library in Python.
- **numpy**: A library for numerical computing in Python.
- **sklearn.metrics.pairwise.cosine_similarity**: A function from scikit-learn used to compute the cosine similarity between vectors.

In [32]:
import pandas as pd
import numpy as np

# Convert userId column to integer type if needed
if df['userId'].dtype != int:
    df['userId'] = df['userId'].astype(int)

# Get all unique user IDs and movie IDs
all_user_ids = np.unique(df['userId'])
all_movie_ids = np.unique(df['movieId'])

# Create a DataFrame with all combinations of user IDs and movie IDs
all_user_movie_pairs = np.array(np.meshgrid(all_user_ids, all_movie_ids)).T.reshape(-1, 2)
df_pairs = pd.DataFrame(all_user_movie_pairs, columns=['userId', 'movieId'])

# Merge the original DataFrame with all_user_movie_pairs to fill missing ratings with 0
df_filled = pd.merge(df_pairs, df, on=['userId', 'movieId'], how='left').fillna(0)

# Create user-item matrix using pivot_table
item_user_matrix = pd.pivot_table(df_filled, values='rating', index='userId', columns='movieId', fill_value=0)

# Convert user-item matrix to NumPy array for faster computation
item_user_array = item_user_matrix.to_numpy()


In [33]:
item_user_matrix.head()

movieId,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
userId,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1333,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
item_user_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Train-test split

The train-val-test split is a technique used in machine learning to evaluate the performance of a model. It involves dividing the dataset into three subsets: the training set, the validation set, and the test set.

The training set is used to train the model and optimize its parameters.
The validation set is used to fine-tune the model and select the best hyperparameters.
The test set is used to evaluate the final performance of the model on unseen data.
By using a train-val-test split, we can assess the model's performance on unseen data and ensure that it generalizes well to new examples. It helps prevent overfitting and provides a more reliable estimate of the model's performance in real-world scenarios.

In [35]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
train_val, test = train_test_split(item_user_array, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("Training set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

# We are also going to do the split for the matrix df
# Split the user-item matrix into training and test sets
train_val_matrix, test_matrix = train_test_split(item_user_matrix, test_size=0.2, random_state=42)

# Split the training set matrix into training and validation sets
train_matrix, val_matrix = train_test_split(train_val_matrix, test_size=0.2, random_state=42)

# We print a ' ' to give some space inbetween lines
print(' ')

# Print the shapes of the matrix datasets
print("Training set matrix shape:", train_matrix.shape)
print("Validation set matrix shape:", val_matrix.shape)
print("Test set matrix shape:", test_matrix.shape)


Training set shape: (10903, 139)
Validation set shape: (2726, 139)
Test set shape: (3408, 139)
 
Training set matrix shape: (10903, 139)
Validation set matrix shape: (2726, 139)
Test set matrix shape: (3408, 139)


In [36]:
train_matrix

movieId,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
userId,,,,,,,,,,,,,,,,,,,,,
2634074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1875533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
2161461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2557296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
881873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2501833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0


In [37]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
# Calculate cosine similarity between items using NumPy functions
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Calculate item-item similarity matrix for train data
#item_similarity_matrix_train = np.zeros((train.shape[1], train.shape[1]))

#for i in range(train.shape[1]):
 #   for j in range(i, train.shape[1]):
  #      item_similarity_matrix_train[i, j] = cosine_similarity(train[:, i], train[:, j])
   #     item_similarity_matrix_train[j, i] = item_similarity_matrix_train[i, j]

# Adjusting the mapping to start indexing from 0
user_id_to_index = {user_id: i for i, user_id in enumerate(item_user_matrix.index)}
index_to_user_id = {i: user_id for i, user_id in enumerate(item_user_matrix.index)}

# Create a mapping from movie IDs to indices
movie_id_to_index = {movie_id: i for i, movie_id in enumerate(item_user_matrix.columns)}
index_to_movie_id = {i: movie_id for i, movie_id in enumerate(item_user_matrix.columns)}




### Similarity Calculation utilizing Genres:



In [39]:
import pandas as pd

# Step 1: Create item-genre matrix
def create_item_genre_matrix(df, train):
    # Extract genre columns from the dataframe
    genre_columns = [col for col in df.columns if col.startswith('genre_')]
    
    # Initialize item-genre matrix with zeros
    item_genre_matrix = pd.DataFrame(0, index=genre_columns, columns=train.columns)
    
    # Fill the matrix with genre information
    for i, row in df.iterrows():
        
        item_id = row['movieId']  # 'movieId' is the column containing item IDs
        genres = row[genre_columns].values
        item_genre_matrix[item_id] = genres
    
    return item_genre_matrix


# df is the original dataframe with genre information and train_matrix is our train set dataframe
item_genre_matrix = create_item_genre_matrix(df, train_matrix)


In [40]:
item_genre_matrix

item_genre_matrix.shape

movieId,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
genre_Action,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
genre_Adventure,0,0,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
genre_Animation,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1
genre_Biography,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
genre_Comedy,1,1,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,1
genre_Crime,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
genre_Documentary,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
genre_Drama,1,0,1,1,1,0,0,1,0,0,...,1,0,1,1,1,1,0,0,0,0
genre_Family,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1
genre_Fantasy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


(22, 139)

#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity. Collaborative filtering is a technique commonly used in recommendation systems to predict a user's preferences for items based on the preferences of similar users/items.


In [41]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_item_similarity_matrix(data):
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(data.T)
    np.fill_diagonal(item_similarity_matrix, 1)  # Set diagonal values to 1
    return item_similarity_matrix

# 'train' is the train set data
item_similarity_matrix_train = calculate_item_similarity_matrix(train)
print(item_similarity_matrix_train)


[[1.         0.00191697 0.00763449 ... 0.         0.00135281 0.01144877]
 [0.00191697 1.         0.         ... 0.         0.         0.        ]
 [0.00763449 0.         1.         ... 0.         0.         0.01330148]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.00135281 0.         0.         ... 0.         1.         0.        ]
 [0.01144877 0.         0.01330148 ... 0.         0.         1.        ]]


In [42]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_item_similarity_genres(item_genre_matrix):
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(item_genre_matrix.T)
    np.fill_diagonal(item_similarity_matrix, 1)  # Set diagonal values to 1
    return item_similarity_matrix

# item_genre_matrix is the item-genre matrix
item_similarity_genres_matrix = calculate_item_similarity_genres(item_genre_matrix)


In [43]:
item_similarity_matrix_train

array([[1.        , 0.00191697, 0.00763449, ..., 0.        , 0.00135281,
        0.01144877],
       [0.00191697, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00763449, 0.        , 1.        , ..., 0.        , 0.        ,
        0.01330148],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.00135281, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.01144877, 0.        , 0.01330148, ..., 0.        , 0.        ,
        1.        ]])

In [44]:
item_similarity_genres_matrix

array([[1.        , 0.40824829, 0.57735027, ..., 0.        , 0.        ,
        0.25819889],
       [0.40824829, 1.        , 0.        , ..., 0.        , 0.        ,
        0.63245553],
       [0.57735027, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.25819889, 0.63245553, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [45]:
movie_id_to_index

{30: 0,
 33: 1,
 44: 2,
 45: 3,
 50: 4,
 52: 5,
 66: 6,
 68: 7,
 71: 8,
 77: 9,
 79: 10,
 102: 11,
 116: 12,
 124: 13,
 164: 14,
 169: 15,
 185: 16,
 199: 17,
 209: 18,
 219: 19,
 221: 20,
 232: 21,
 234: 22,
 238: 23,
 240: 24,
 248: 25,
 271: 26,
 290: 27,
 298: 28,
 310: 29,
 311: 30,
 333: 31,
 340: 32,
 345: 33,
 352: 34,
 354: 35,
 355: 36,
 359: 37,
 4506: 38,
 4509: 39,
 4513: 40,
 4521: 41,
 4554: 42,
 4571: 43,
 4578: 44,
 4592: 45,
 4600: 46,
 4601: 47,
 4610: 48,
 4625: 49,
 4636: 50,
 4666: 51,
 4667: 52,
 4671: 53,
 4693: 54,
 4705: 55,
 4727: 56,
 4729: 57,
 4730: 58,
 4733: 59,
 4738: 60,
 4744: 61,
 4746: 62,
 4752: 63,
 4758: 64,
 4761: 65,
 4768: 66,
 4777: 67,
 4788: 68,
 4790: 69,
 4805: 70,
 9214: 71,
 9218: 72,
 9233: 73,
 9240: 74,
 9249: 75,
 9255: 76,
 9270: 77,
 9305: 78,
 9311: 79,
 9317: 80,
 9319: 81,
 9329: 82,
 9331: 83,
 9339: 84,
 9348: 85,
 9367: 86,
 9368: 87,
 9372: 88,
 9385: 89,
 9389: 90,
 9390: 91,
 9403: 92,
 9412: 93,
 9436: 94,
 9441: 95,
 94

In [46]:
print(index_to_user_id)

{0: 7, 1: 596, 2: 609, 3: 1333, 4: 1427, 5: 1588, 6: 1715, 7: 1830, 8: 1871, 9: 2128, 10: 2213, 11: 2225, 12: 2455, 13: 2469, 14: 2960, 15: 2976, 16: 3039, 17: 3321, 18: 3363, 19: 3694, 20: 3788, 21: 4277, 22: 4306, 23: 4421, 24: 4576, 25: 4597, 26: 4679, 27: 4706, 28: 4753, 29: 4905, 30: 4906, 31: 4983, 32: 5399, 33: 5727, 34: 5980, 35: 6206, 36: 6504, 37: 6529, 38: 6736, 39: 6815, 40: 7581, 41: 7601, 42: 7730, 43: 7921, 44: 8121, 45: 8200, 46: 8212, 47: 8367, 48: 8660, 49: 8797, 50: 9321, 51: 9557, 52: 9660, 53: 9691, 54: 10161, 55: 10268, 56: 10374, 57: 10380, 58: 10574, 59: 10679, 60: 10805, 61: 10901, 62: 10943, 63: 10977, 64: 11043, 65: 11059, 66: 11169, 67: 11215, 68: 11310, 69: 11398, 70: 11430, 71: 11462, 72: 11598, 73: 11734, 74: 11796, 75: 12138, 76: 12168, 77: 12253, 78: 12798, 79: 12876, 80: 13227, 81: 13266, 82: 13333, 83: 13432, 84: 13528, 85: 13586, 86: 13620, 87: 13843, 88: 13891, 89: 14081, 90: 14164, 91: 14389, 92: 14704, 93: 14936, 94: 15027, 95: 15042, 96: 15065, 9

In [47]:
print(user_id_to_index)

{7: 0, 596: 1, 609: 2, 1333: 3, 1427: 4, 1588: 5, 1715: 6, 1830: 7, 1871: 8, 2128: 9, 2213: 10, 2225: 11, 2455: 12, 2469: 13, 2960: 14, 2976: 15, 3039: 16, 3321: 17, 3363: 18, 3694: 19, 3788: 20, 4277: 21, 4306: 22, 4421: 23, 4576: 24, 4597: 25, 4679: 26, 4706: 27, 4753: 28, 4905: 29, 4906: 30, 4983: 31, 5399: 32, 5727: 33, 5980: 34, 6206: 35, 6504: 36, 6529: 37, 6736: 38, 6815: 39, 7581: 40, 7601: 41, 7730: 42, 7921: 43, 8121: 44, 8200: 45, 8212: 46, 8367: 47, 8660: 48, 8797: 49, 9321: 50, 9557: 51, 9660: 52, 9691: 53, 10161: 54, 10268: 55, 10374: 56, 10380: 57, 10574: 58, 10679: 59, 10805: 60, 10901: 61, 10943: 62, 10977: 63, 11043: 64, 11059: 65, 11169: 66, 11215: 67, 11310: 68, 11398: 69, 11430: 70, 11462: 71, 11598: 72, 11734: 73, 11796: 74, 12138: 75, 12168: 76, 12253: 77, 12798: 78, 12876: 79, 13227: 80, 13266: 81, 13333: 82, 13432: 83, 13528: 84, 13586: 85, 13620: 86, 13843: 87, 13891: 88, 14081: 89, 14164: 90, 14389: 91, 14704: 92, 14936: 93, 15027: 94, 15042: 95, 15065: 96, 1

In [48]:
item_user_array.shape

(17037, 139)

In [49]:
item_user_matrix.shape

(17037, 139)

In [50]:
# We print the item_similarity_matrix
item_similarity_matrix_train

array([[1.        , 0.00191697, 0.00763449, ..., 0.        , 0.00135281,
        0.01144877],
       [0.00191697, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00763449, 0.        , 1.        , ..., 0.        , 0.        ,
        0.01330148],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.00135281, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.01144877, 0.        , 0.01330148, ..., 0.        , 0.        ,
        1.        ]])

In [51]:
item_similarity_matrix_train.shape

(139, 139)

### Matrices Summation:

Once we have both similarity matrices (the one from the ratings and the one from the genres) we can add them with a weighted sumation.

In [52]:
def combine_similarity_matrices(similarity_matrix1, similarity_matrix2, weight1 = 0.7, weight2 = 0.3):
    # Check if the matrices have the same shape
    if similarity_matrix1.shape != similarity_matrix2.shape:
        raise ValueError("Matrices must have the same shape.")

    # Combine matrices using weighted summation
    combined_similarity_matrix = (weight1 * similarity_matrix1) + (weight2 * similarity_matrix2)
    return combined_similarity_matrix

# Define weights for each matrix
weight_ratings = 0.7
weight_genres = 0.3

# Combine the similarity matrices
item_similarity_matrix_train = combine_similarity_matrices(item_similarity_matrix_train, item_similarity_genres_matrix, weight_ratings, weight_genres)

# Print or use the combined similarity matrix as needed
print("Combined Similarity Matrix:\n", item_similarity_matrix_train)


Combined Similarity Matrix:
 [[1.00000000e+00 1.23816366e-01 1.78549225e-01 ... 0.00000000e+00
  9.46966785e-04 8.54738037e-02]
 [1.23816366e-01 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.89736660e-01]
 [1.78549225e-01 0.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 9.31103265e-03]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.46966785e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [8.54738037e-02 1.89736660e-01 9.31103265e-03 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]]


### Step 2: Neighborhood Selection
- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in our dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `neighborhood_size` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.
- The choice of rating aggregation method (e.g., mean, median) can impact the quality of recommendations and may need to be adjusted based on the characteristics of the dataset and user preferences.


In [53]:
import numpy as np

def neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix, neighborhood_size=5):
    # Initialize an empty dictionary to store item neighborhoods
    item_neighborhoods = {}

    # Iterate over each item (movie) index in the dataset
    for movie_index in range(item_user_array.shape[1]):
        # Convert the item index to movie ID
        movie_id = index_to_movie_id[movie_index]

        # Extract all ratings for the current movie
        movie_ratings = item_user_array[:, movie_index]

        # Aggregate ratings (e.g., compute the mean rating)
        movie_avg_rating = np.mean(movie_ratings)

        # Retrieve similarity scores for the current movie
        similarity_scores = item_similarity_matrix[movie_index]

        # Sort similarity scores in descending order and get indices of most similar items
        most_similar_indices = np.argsort(similarity_scores)[::-1][1:neighborhood_size+1]

        # Convert indices back to movie IDs to form the neighborhood
        neighborhood = [index_to_movie_id[idx] for idx in most_similar_indices]

        # Store the neighborhood for the current item in the item_neighborhoods dictionary
        item_neighborhoods[movie_id] = neighborhood

    return item_neighborhoods

# Example usage:
item_neighborhoods_train = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train)
print(item_neighborhoods_train)


{30: [4506, 45, 4790, 13625, 352], 33: [9319, 13581, 4625, 13673, 352], 44: [232, 9317, 4729, 102, 9331], 45: [4506, 30, 13625, 352, 4790], 50: [352, 4768, 9311, 199, 9550], 52: [9552, 9305, 4733, 9249, 199], 66: [9319, 333, 13543, 13468, 13576], 68: [9348, 13673, 209, 13478, 290], 71: [9305, 4610, 9249, 9403, 9233], 77: [209, 9348, 13478, 13458, 68], 79: [13596, 13409, 124, 4805, 354], 102: [4746, 44, 9317, 9331, 232], 116: [4790, 4601, 45, 311, 352], 124: [13409, 13596, 9233, 79, 13576], 164: [13383, 13626, 9550, 30, 45], 169: [4667, 185, 333, 4578, 4625], 185: [169, 4667, 9233, 271, 30], 199: [4727, 9311, 9317, 44, 9331], 209: [77, 9348, 13478, 68, 13458], 219: [4746, 44, 9317, 9331, 102], 221: [13478, 4738, 9603, 116, 13489], 232: [44, 9317, 219, 9331, 102], 234: [13673, 4571, 4610, 4601, 4600], 238: [4744, 9249, 4733, 4554, 13497], 240: [4506, 4790, 30, 352, 4601], 248: [9331, 13625, 4752, 9469, 9368], 271: [340, 9603, 13458, 185, 4752], 290: [13478, 4625, 9319, 13673, 4705], 298:

### Step 4: Rating Prediction

- For each target item and user pair where the user hasn't rated the target item:
Identify the neighborhood of similar items to the target item.
- Predict the rating for the target item using a weighted average of the ratings of the items in its neighborhood, where the weights are the similarities between the items and the target item.
- Adjust the prediction based on the user's average rating or other normalization techniques, if necessary.

In [54]:
# Train our model and create the item_neighborhoods dictionary using the training data (steps 1-3)

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_train = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in train_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_train.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)

# Display the head of the predicted_ratings DataFrame
predicted_ratings_train.head()


KeyboardInterrupt: 

In [ ]:
# We count the NaN values in the predicted_ratings_train dataframe
predicted_ratings_train.isnull().sum().sum()

0

In [ ]:
def get_top_recommendations(user_id, predicted_ratings, df):
    """
    Get top movie recommendations for a given user using predicted ratings.

    Parameters:
        user_id (int): ID of the user for whom recommendations are to be generated.
        predicted_ratings (pd.DataFrame): DataFrame containing predicted ratings for users and movies.
        df (pd.DataFrame): DataFrame containing movie ratings.

    Returns:
        list: Top movie titles recommended for the user.
    """
    # Ensure that predicted_ratings is a DataFrame
    if not isinstance(predicted_ratings, pd.DataFrame):
        raise ValueError("predicted_ratings must be a pandas DataFrame")

    # Check if the user ID exists in the predicted ratings DataFrame's index
    if user_id not in predicted_ratings.index:
        # If the user ID doesn't exist, return an empty list
        return ["The user doesn't exist"]

    # Get the predicted ratings for the user
    user_predicted_ratings = predicted_ratings.loc[user_id]
    
    # Filter out the movies that the user has already seen
    seen_movies = set(df[df['userId'] == user_id]['movieId'])
    unseen_movies = [movie_id for movie_id in user_predicted_ratings.index if movie_id not in seen_movies]
    
    # Check if there are unseen movies with predicted ratings
    if not unseen_movies:
        # If all movies have been seen, return an empty list
        return []
    
    # Sort the unseen movies by predicted rating in descending order
    sorted_unseen_movies = user_predicted_ratings[unseen_movies].sort_values(ascending=False)
    
    # Get the top 5 movie titles
    top_movie_ids = sorted_unseen_movies.head(5).index.tolist()
    
    # Get the unique movie titles corresponding to the top movie IDs
    top_movie_titles = set(df[df['movieId'].isin(top_movie_ids)]['title'])
    
    return list(top_movie_titles)[:5]  # Return only the first 5 unique movie titles


In [ ]:
top_recommendations = []

# Iterate over each row (user) in the train_matrix
for user_id in train_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_train, df)
    top_recommendations.append(recommendations)

# Output: List of top movie recommendations for each user in the train matrix
top_recommendations

# print size of top_recommendations
len(top_recommendations)


[['Colonel Wolodyjowski',
  'Perfect',
  'The Texas Chainsaw Massacre',
  'Pacific Heights',
  'Led Zeppelin: The Song Remains the Same'],
 ['In July',
  'Welcome to Mooseport',
  "Something's Gotta Give",
  'Into the West',
  'I.M. Pei'],
 ['Overboard',
  'Rumpole of the Bailey: The Lost Episode',
  "Something's Gotta Give",
  'Lady Jane',
  'Ed Wood'],
 ['Sadomania',
  'West Hollywood Stories',
  'Legend of Crystania: The Chaos Ring',
  'Absolutely Fabulous: Series 5',
  'Red Lights'],
 ['In July',
  'Welcome to Mooseport',
  "Something's Gotta Give",
  'Like Father, Like Son',
  'Into the West'],
 ['In July',
  'Welcome to Mooseport',
  "Something's Gotta Give",
  'Like Father, Like Son',
  'Into the West'],
 ['In July',
  'Welcome to Mooseport',
  "Something's Gotta Give",
  'Like Father, Like Son',
  'Into the West'],
 ['Cooley High',
  "Something's Gotta Give",
  'Mr. Jones',
  'Lord of the Flies',
  'Cartoon Network Halloween: 9 Creepy Cartoon Capers'],
 ['The Big O',
  'Missing

26069

In [ ]:
print("Last version")

Last version


### Step 5: Model Evaluation
- We evaluate the performance of our ItemKNN algorithm using appropriate evaluation metrics such as Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), or others.
- Split our dataset into training and testing sets to assess the model's predictive accuracy on unseen data.

In [ ]:
train.shape

predicted_ratings_train.shape

(26069, 134)

(26069, 134)

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(train_matrix, predicted_ratings):
    """
    Evaluate the model's performance on the training data.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        predicted_ratings (pandas.DataFrame or numpy.ndarray): Predicted ratings DataFrame or array for the training data.

    Returns:
        float: Mean Absolute Error (MAE) on the training data.
        float: Root Mean Squared Error (RMSE) on the training data.
    """
    # Convert predicted_ratings to a numpy array if it's a DataFrame
    if isinstance(predicted_ratings, pd.DataFrame):
        predicted_ratings = predicted_ratings.to_numpy()

    # Ensure train_matrix and predicted_ratings have the same shape
    assert train_matrix.shape == predicted_ratings.shape, "Shapes of train_matrix and predicted_ratings are not consistent."

    # Initialize lists to store true and predicted ratings
    true_ratings = []
    pred_ratings = []

    # Iterate over each user and their ratings
    for user_id, user_ratings in enumerate(train_matrix):
        for movie_id, rating in enumerate(user_ratings):
            # Skip unrated movies
            if rating == 0:
                continue

            # Check if the indices are within the bounds of the predicted ratings array
            if user_id < predicted_ratings.shape[0] and movie_id < predicted_ratings.shape[1]:
                # Get the predicted rating for the corresponding movie
                pred_rating = predicted_ratings[user_id, movie_id]

                # Append the true and predicted ratings
                true_ratings.append(rating)
                pred_ratings.append(pred_rating)

    # Convert lists to numpy arrays
    true_ratings = np.array(true_ratings)
    pred_ratings = np.array(pred_ratings)

    # Calculate evaluation metrics
    mae = mean_absolute_error(true_ratings, pred_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, pred_ratings))

    return mae, rmse

# train is the numpy array representing the item-user matrix
# predicted_ratings_train is the DataFrame or numpy array representing the predicted ratings for the training data

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.908365605569637
Root Mean Squared Error (RMSE) on Training Data: 1.0839196962227091


### Step 6: Parameter Tuning
- Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of our ItemKNN algorithm.
- Use cross-validation or other techniques to tune these parameters and avoid overfitting.

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



`scikit-learn (sklearn)`:

- Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data analysis and modeling.
- It includes various modules for tasks such as classification, regression, clustering, dimensionality reduction, and model selection.
- The GridSearchCV class from scikit-learn is used for hyperparameter tuning through grid search along with cross-validation.
- The make_scorer function allows us to create a custom scoring function for use with GridSearchCV.
- The NearestNeighbors class provides functionality for unsupervised nearest neighbors learning, which can be used for tasks such as finding k-nearest neighbors for a given data point.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # train_matrix contains item-user matrix for the train set

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


This code sets up a GridSearchCV object to perform hyperparameter tuning using the train set (train_matrix). It explores different combinations of hyperparameters specified in the param_grid, evaluates them using 5-fold cross-validation (cv=5), and uses the cosine_similarity scorer to optimize the model's performance based on cosine similarity. Finally, it prints the best hyperparameters found during the search.

We have to perform the calculation of the predicted ratings for the train-val set to generate predicted ratings in order to evaluate the performance of our trained model on a validation dataset. 

If we computed the similarity matrix again specifically for the train_val set, it would essentially mean that we are using a different set of similarity measures for predicting ratings compared to what we used during training. This approach could lead to inconsistencies and potentially degrade the performance of our model. Here's what could happen:

1. **Inconsistency**: The similarity measures computed for the train_val set might differ from those computed for the training set due to variations in the data. As a result, the predicted ratings based on these new similarity measures may not align well with the predictions made during training, leading to inconsistency in the model's behavior.

2. **Overfitting**: Computing a new similarity matrix specifically for the train_val set might lead to overfitting on the validation data. The model may capture noise or idiosyncrasies present in the train_val set, which may not generalize well to unseen data.

3. **Increased Complexity**: Computing the similarity matrix again for the train_val set adds computational complexity and redundancy, especially if the similarity computation process is resource-intensive. This can result in longer training times and increased resource utilization.

Overall, it's generally recommended to use the same similarity measures or neighborhood definitions for both training and validation sets to ensure consistency and generalizability of the model.

In [ ]:
# Initialize an empty DataFrame to store predicted ratings for train_val
predicted_ratings_val = pd.DataFrame(index=val_matrix.index, columns=val_matrix.columns)

# Iterate over each user and their ratings in the train_val set
for user_id, user_ratings in val_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_val.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_1620\2864728636.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


In [ ]:
predicted_ratings_val.head()

predicted_ratings_val.shape

movieId,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
userId,,,,,,,,,,,,,,,,,,,,,
1586084,3.585463,4.00,3.0,4.000000,4.00,4.50,4.00,4.00,4.00,3.0,...,4.00,4.0,4.0,4.00,3.0,4.00,3.500000,4.00,4.00,3.0
40450,3.585463,4.00,4.0,4.000000,4.00,4.00,4.00,4.00,4.00,4.0,...,4.00,4.0,4.0,4.00,4.0,4.00,4.000000,4.00,4.00,4.0
1862392,2.000000,2.00,2.0,2.000000,2.00,2.00,2.00,2.00,2.00,2.0,...,2.00,2.0,2.0,2.00,2.0,2.00,2.000000,2.00,2.00,2.0
2111228,4.000000,4.00,4.0,4.000000,4.00,4.00,4.00,4.00,4.00,4.0,...,4.00,4.0,4.0,4.00,4.0,4.00,4.000000,4.00,4.00,4.0
1216438,3.585463,4.25,4.5,3.585463,4.25,4.25,4.25,4.25,4.25,4.0,...,4.25,4.0,4.0,4.25,4.0,4.25,3.585463,4.25,4.25,4.0


(6518, 134)

Now we evaluate the model with the Validation item-user matrix.

In [ ]:
# Now, we can evaluate the model using the evaluate_model function
mae, rmse = evaluate_model(val_matrix.to_numpy(), predicted_ratings_val)

print("Mean Absolute Error (MAE) on Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Validation Data:", rmse)


Mean Absolute Error (MAE) on Validation Data: 0.8925291182030209
Root Mean Squared Error (RMSE) on Validation Data: 1.0638545993011213


### Retraining of the model with the Train-Val set

In [ ]:
# After the model evaluation on the validation set, we recalculate the similarity matrix
# of the model on the train-val data
item_similarity_matrix_train_val = calculate_item_similarity_matrix(train_val)

# We combine the similarity matrices (the retrained one and the genres)
item_similarity_matrix_train_val = combine_similarity_matrices(item_similarity_matrix_train_val, item_similarity_genres_matrix)

# We calculate the item neighborhoods for the train_val data 
item_neighborhoods_train_val = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train_val)

### Step 7: Deployment

Once we have come up with the best parameters possible and trained the model with the whole train_validation set, we will test it on the test set.

In [ ]:
test_matrix

movieId,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
userId,,,,,,,,,,,,,,,,,,,,,
1210904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
854332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1961848,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1450048,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017543,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2426539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
705960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Check if there are NaN values in the test matrix
test_matrix.isnull().sum().sum()

0

In [ ]:
# Now, we can evaluate the model's predicted ratings on the test set

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_test = pd.DataFrame(index=test_matrix.index, columns=test_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in test_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Skip if the rating is non-zero (indicating a rating given by the user)
        if rating != 0:
            continue
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train_val:
            neighborhood = item_neighborhoods_train_val[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_test.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)

# Now, we can evaluate the model's predicted ratings on the test set
mae, rmse = evaluate_model(test_matrix.to_numpy(), predicted_ratings_test.to_numpy())

print("Mean Absolute Error (MAE) on Test Data using Predicted Ratings from Test Set:", mae)
print("Root Mean Squared Error (RMSE) on Test Data using Predicted Ratings from Test Set:", rmse)


Mean Absolute Error (MAE) on Test Data using Predicted Ratings from Test Set: 0.8958318165803375
Root Mean Squared Error (RMSE) on Test Data using Predicted Ratings from Test Set: 1.0680886618656604


C:\Users\Jaume\AppData\Local\Temp\ipykernel_1620\912451769.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)


In [ ]:
top_recommendations_test = []

# Iterate over each row (user) in the test_matrix
for user_id in test_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_test, df)
    top_recommendations_test.append(recommendations)

# Output: List of top movie recommendations for each user in the test matrix
top_recommendations_test

# print size of top_recommendations
print("Size of top_recommendations_test:", len(top_recommendations_test))



[['Dennis the Menace',
  'Welcome to Mooseport',
  'Blue Streak',
  'Massage Your Mate',
  'Kazaam'],
 ['Full Metal Panic FUMOFFU',
  'Legend of Crystania: The Chaos Ring',
  'Bear Cub',
  'Spitfire Grill',
  'Absolutely Fabulous: Series 5'],
 ['In July',
  'Welcome to Mooseport',
  "Something's Gotta Give",
  'Like Father, Like Son',
  'Into the West'],
 ['Dressed to Kill',
  'Leprechaun 2',
  'Harold and Kumar Go to White Castle',
  'Videodrome',
  'Welcome to Mooseport'],
 ['In July',
  'Welcome to Mooseport',
  'Aqua Teen Hunger Force: Vol. 1',
  'Like Father, Like Son',
  'Into the West'],
 ['In July',
  'Welcome to Mooseport',
  "Something's Gotta Give",
  'Like Father, Like Son',
  'Into the West'],
 ['Perfect',
  'Lord of the Flies',
  'Pacific Heights',
  "Breakfast at Tiffany's",
  'Alice in Wonderland'],
 ['In July',
  'Welcome to Mooseport',
  "Something's Gotta Give",
  'Like Father, Like Son',
  'Into the West'],
 ['The Big O',
  'Welcome to Mooseport',
  "Something's Got

Size of top_recommendations_test: 8147


# Item-based Classification (ItemKNN)

### Step 1: Data preparation
- Ensure we have a dataset with user-item interactions. Each interaction should include the user ID, item ID, and the corresponding label or class for classification.
- Preprocess the data as needed, including handling missing values, encoding categorical variables, and splitting into training and testing sets.

For this step we are going to resuse the `item_user_matrix` that we created in the first model built. 

### Step 2: Compute Item Similarity


In [ ]:
item_similarity_matrix_train.shape

(134, 134)

In [ ]:
import numpy as np

def item_neighborhood_selection(similarity_matrix, k=None, threshold=None, item_ids=None):
    """
    Select a subset of similar items for each item based on similarity matrix.

    Parameters:
        similarity_matrix (numpy.ndarray): Item-item similarity matrix.
        k (int): Number of similar items to select (optional).
        threshold (float): Similarity threshold for selecting similar items (optional).
        item_ids (list): List of item IDs corresponding to rows/columns of the similarity matrix.

    Returns:
        dict: Dictionary containing similar items for each item.
    """
    num_items = similarity_matrix.shape[0]
    item_neighborhood = {}

    for i in range(num_items):
        if k is not None:
            # Select top-k similar items (excluding the item itself)
            similar_items_indices = np.argsort(similarity_matrix[i])[::-1][:k]
        elif threshold is not None:
            # Select items with similarity above threshold (excluding the item itself)
            similar_items_indices = np.where(similarity_matrix[i] > threshold)[0]

        # Remove the item itself from the neighborhood
        similar_items_indices = similar_items_indices[similar_items_indices != i]

        # Get the item ID corresponding to the current index
        current_item_id = item_ids[i] if item_ids is not None else i

        # Get the item IDs for the neighborhood
        neighborhood_item_ids = [item_ids[index] for index in similar_items_indices]

        # Store the similar items with the item's ID as the key
        item_neighborhood[current_item_id] = neighborhood_item_ids

    return item_neighborhood

# We have an item-item similarity matrix 'item_similarity_matrix_train'
# and we want to select top-5 similar items for each item
k = 5
item_neighborhoods_classification_train = item_neighborhood_selection(item_similarity_matrix_train, k=k, item_ids=index_to_movie_id)

# Check if there are any items with empty neighborhoods
empty_neighborhoods = []

for item_id, neighborhood in item_neighborhoods_classification_train.items():
    if not neighborhood:
        empty_neighborhoods.append(item_id)

if empty_neighborhoods:
    print("Items with empty neighborhoods:", empty_neighborhoods)
else:
    print("No items with empty neighborhoods found.")


No items with empty neighborhoods found.


In [ ]:
len(item_neighborhoods_classification_train)

134

### Step 3: Ratings classification selection

In [ ]:
import numpy as np
import pandas as pd

def recommend_movies(train_matrix, item_neighborhoods, movie_id_to_index, num_movies=5):
    """
    Recommend top movies for each user based on item neighborhood.

    Parameters:
        train_matrix (np.ndarray or pd.DataFrame): Matrix containing user-item ratings in the train set.
        item_neighborhoods (dict): Dictionary containing similar items for each item.
        movie_id_to_index (dict): Dictionary mapping movie IDs to their corresponding indices in the train_matrix.
        num_movies (int): Number of movies to recommend for each user.

    Returns:
        pd.DataFrame: DataFrame containing predicted ratings for recommended movies for each user.
    """
    # Convert train_matrix to DataFrame if it's a numpy array
    if isinstance(train_matrix, np.ndarray):
        train_matrix = pd.DataFrame(train_matrix, index=np.arange(train_matrix.shape[0]), columns=np.arange(train_matrix.shape[1]))

    # Initialize an empty DataFrame to store predicted ratings
    predicted_ratings = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

    # Iterate over each user-item pair in the training set
    for user_id, user_ratings in train_matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]
                
                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False
                
                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break
                
                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = train_matrix[train_matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()
                
                # Assign the predicted rating to the corresponding cell in the DataFrame
                predicted_ratings.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)

    # Set the index of the DataFrame as the user ID of the train set
    predicted_ratings.index = train_matrix.index

    # Display the head of the predicted_ratings DataFrame
    print(predicted_ratings.head())

    return predicted_ratings

# Recommend movies for each user in the train set
predicted_ratings_classification_train = recommend_movies(train_matrix, item_neighborhoods_classification_train, movie_id_to_index)

# Print the shape of the predicted ratings matrix
print("Shape of predicted ratings matrix:", predicted_ratings_classification_train.shape)


In [ ]:
type(train_matrix)

pandas.core.frame.DataFrame

In [ ]:
predicted_ratings_classification_train

movieId,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
userId,,,,,,,,,,,,,,,,,,,,,
1808379,4.00,4.00,4.000000,4.00,4.00,4.00,4.0,4.00,4.00,4.000000,...,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.000000
1187089,2.00,2.00,2.000000,2.00,2.00,2.00,2.0,2.00,2.00,3.555001,...,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.000000
1707198,3.00,3.50,3.500000,3.50,3.50,3.50,3.5,3.50,3.50,3.500000,...,3.00,3.50,3.50,4.00,3.50,3.50,3.50,3.50,3.50,3.500000
2211057,3.00,3.00,3.000000,3.00,3.00,3.00,3.0,3.00,3.00,3.000000,...,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.000000
195032,3.00,3.00,3.000000,3.00,3.00,3.00,3.0,3.00,3.00,3.000000,...,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252784,4.00,4.00,4.000000,4.00,4.00,4.00,4.0,4.00,4.00,4.000000,...,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.000000
1280428,4.00,4.00,3.555001,4.00,4.00,4.00,4.0,4.00,4.00,4.000000,...,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.000000
196349,3.00,3.00,3.000000,3.00,3.00,3.00,3.0,3.00,3.00,3.000000,...,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.000000


### Step 4: Recommendations Generation

In [ ]:
import pandas as pd

# Convert predicted_ratings_classification_train to a pandas DataFrame if it's a numpy array
if isinstance(predicted_ratings_classification_train, np.ndarray):
    # Use the length of train_matrix as the number of rows and items as columns
    num_users, num_items = train_matrix.shape
    predicted_ratings_classification_train = pd.DataFrame(predicted_ratings_classification_train, index=range(num_users), columns=range(num_items))

# Initialize a dictionary to store top recommendations for each user
top_recommendations_per_user = {}

# Iterate over each user in the train matrix
for user_id in train_matrix.index:
    # Get the top recommendations for the current user using predicted_ratings_classification_train
    recommendations = get_top_recommendations(user_id, predicted_ratings_classification_train, df)
    
    # Store the recommendations in the dictionary
    top_recommendations_per_user[user_id] = recommendations

# Print the top 5 recommendations for each user
for user_id, recommendations in top_recommendations_per_user.items():
    print(f"User {user_id} recommendations:", recommendations)


User 1808379 recommendations: ['Welcome to Mooseport', 'The Big O', "Something's Gotta Give", 'Cold Mountain: Bonus Material', 'Into the West']
User 1187089 recommendations: ['Welcome to Mooseport', 'The Big O', "Something's Gotta Give", 'Cold Mountain: Bonus Material', 'Into the West']
User 1707198 recommendations: ['Stevie', 'If These Walls Could Talk 2', 'True Romance', 'Monument Ave.', 'Red Lights']
User 2211057 recommendations: ['Welcome to Mooseport', 'Slow Burn', "I'll Sleep When I'm Dead", "Something's Gotta Give", 'Cold Mountain: Bonus Material']
User 195032 recommendations: ['Welcome to Mooseport', 'The Big O', "Something's Gotta Give", 'Cold Mountain: Bonus Material', 'Into the West']
User 1202268 recommendations: ['Welcome to Mooseport', 'The Big O', "Something's Gotta Give", 'Cold Mountain: Bonus Material', 'Into the West']
User 2509631 recommendations: ['Welcome to Mooseport', 'Slow Burn', "I'll Sleep When I'm Dead", "Something's Gotta Give", 'Cold Mountain: Bonus Materia

### Step 5: Model Evaluation (Item KNN Classification)

In [ ]:
import numpy as np
import pandas as pd

# predicted_ratings_classification_train is our predicted ratings DataFrame or numpy array

# Ensure predicted_ratings_classification_train is a numpy array
if isinstance(predicted_ratings_classification_train, pd.DataFrame):
    predicted_ratings_classification_train = predicted_ratings_classification_train.to_numpy()

# train_matrix is the numpy array representing the item-user matrix

# Ensure train_matrix is a 2D array
if isinstance(train_matrix, pd.DataFrame):
    train_matrix = train_matrix.to_numpy()

if train_matrix.ndim == 1:
    train_matrix = np.expand_dims(train_matrix, axis=0)

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_classification_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.899138240564858
Root Mean Squared Error (RMSE) on Training Data: 1.068985357745771


### Step 6: Parameter Tuning

Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of our ItemKNN algorithm.
Use techniques like cross-validation to tune these parameters and avoid overfitting.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(val_matrix)  # val_matrix contains item-user matrix

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


We are now going to obtain again the ratings for the full val set to evaluate the performance. 

In [ ]:
# Function to recalculate predictions for a given matrix using the trained model
def recalculate_predictions(matrix, item_neighborhoods):
    """
    Recalculate predictions for a given matrix using the trained model.

    Parameters:
        matrix (pd.DataFrame): Matrix containing user-item ratings.

    Returns:
        pd.DataFrame: DataFrame containing recalculated predicted ratings for each user.
    """
    # Initialize an empty DataFrame to store recalculated predicted ratings
    recalculated_predictions = pd.DataFrame(index=matrix.index, columns=matrix.columns)

    # Iterate over each user-item pair in the matrix
    for user_id, user_ratings in matrix.iterrows():
        for movie_id, rating in user_ratings.items():
        
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]

                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False

                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break

                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = matrix[matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()

                # Assign the predicted rating to the corresponding cell in the DataFrame
                recalculated_predictions.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)

    return recalculated_predictions


# Recalculate predictions for the validation set (train_val_matrix)
recalculated_val_predictions = recalculate_predictions(val_matrix, item_neighborhoods_classification_train)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_8840\4246365781.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


In [ ]:
recalculated_val_predictions

movieId,30,33,44,45,50,52,66,68,71,77,...,13576,13581,13594,13596,13625,13626,13629,13656,13668,13673
userId,,,,,,,,,,,,,,,,,,,,,
1070896,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
481980,2.000000,2.0,2.0,2.000000,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2245483,3.000000,3.0,3.0,3.000000,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
550341,5.000000,5.0,5.0,5.000000,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2119988,3.561832,4.0,4.0,4.000000,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378784,2.000000,2.0,2.0,3.561832,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
18830,2.000000,2.0,2.0,2.000000,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2013305,4.000000,4.0,4.0,4.000000,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


In [ ]:
# Evaluate the model
mae, rmse = evaluate_model(val, recalculated_val_predictions)

print("Mean Absolute Error (MAE) on Train-Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)

Mean Absolute Error (MAE) on Train-Validation Data: 0.9157890482511658
Root Mean Squared Error (RMSE) on Train-Validation Data: 1.0912481750562741


### Step 7: Deployment

Once we have all the metrics, we have to recalculate the ratings for the test set, we are going to repeat the same process for the test set.

In [ ]:
# We have to reobtain the neighborhood for the train_val data
# We will use the already calculated item-item similarity matrix for train_val data
item_neighborhoods_classification_train_val = item_neighborhood_selection(item_similarity_matrix_train_val, k=5, item_ids=index_to_movie_id)

# Recalculate predictions for the validation set (test matrix)
recalculated_test_predictions = recalculate_predictions(test_matrix, item_neighborhoods_classification_train_val)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_8840\4246365781.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


We also have to evaluate the metrics for the test set.

In [ ]:
# Evaluate the model
mae, rmse = evaluate_model(test, recalculated_test_predictions)

print("Mean Absolute Error (MAE) on Test Data:", mae)
print("Root Mean Squared Error (RMSE) on Train-Validation Data:", rmse)

Mean Absolute Error (MAE) on Test Data: 0.9021627996092461
Root Mean Squared Error (RMSE) on Train-Validation Data: 1.072702296516033
